In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from scipy import signal
import scipy
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from scipy.signal import welch
from scipy.stats import entropy

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
import pywt

def wavelet_denoising(data, wavelet='db4', level=1):
    coeff = pywt.wavedec(data, wavelet, mode="per")
    sigma = (1/0.6745) * np.median(np.abs(coeff[-level] - np.median(coeff[-level])))
    uthresh = sigma * np.sqrt(2 * np.log(len(data)))
    coeff[1:] = (pywt.threshold(i, value=uthresh, mode='soft') for i in coeff[1:])
    return pywt.waverec(coeff, wavelet, mode="per")


In [7]:
import numpy as np
from scipy.signal import find_peaks, butter, filtfilt
from numpy import trapezoid

def extract_ppg_features_per_beat(ppg, fs=100):
    # Preprocessing: Bandpass filter (0.5-8Hz)
    def filter_signal(signal, low=0.5, high=8, fs=100, order=4):
        nyq = 0.5 * fs  # Nyquist frequency
        low_norm = low / nyq  # Normalized low cutoff
        high_norm = high / nyq  # Normalized high cutoff

        # Validate frequency range
        if low_norm <= 0 or high_norm >= 1:
            raise ValueError(f"Filter frequencies must be between 0 and {nyq} Hz for fs={fs}")

        b, a = butter(order, [low_norm, high_norm], btype='band')
        return filtfilt(b, a, signal)

    try:
        ppg_filt = filter_signal(ppg, fs=fs)
    except ValueError as e:
        print(f"Filtering error: {str(e)}")
        return []

    # Detect systolic peaks with more robust parameters
    min_peak_distance = int(0.5 * fs)  # 500ms minimum between peaks
    peaks, _ = find_peaks(ppg_filt,
                         distance=min_peak_distance,
                         prominence=0.3*np.max(ppg_filt))

    if len(peaks) < 3:
        return []  # Insufficient beats

    beat_starts = []
    beat_ends = []

    for i in range(len(peaks)):
        if i == 0:
            left = max(0, peaks[i] - min_peak_distance)
        else:
            left = peaks[i-1] + np.argmin(ppg_filt[peaks[i-1]:peaks[i]])

        if i < len(peaks)-1:
            right = peaks[i] + np.argmin(ppg_filt[peaks[i]:peaks[i+1]])
        else:
            right = min(peaks[i] + min_peak_distance, len(ppg_filt)-1)

        beat_starts.append(left)
        beat_ends.append(right)

    features_list = []

    for i in range(len(peaks)):
        try:
            start, end = beat_starts[i], beat_ends[i]
            beat_seg = ppg_filt[start:end]
            t = np.arange(len(beat_seg)) / fs

            # Find diastolic minimum (foot of the beat)
            diastole = np.argmin(beat_seg)

            # Find systolic peak (relative to segment)
            systole = peaks[i] - start

            post_peak = beat_seg[systole:]
            if len(post_peak) > 10:
                post_peak_smooth = np.convolve(post_peak, np.ones(5)/5, mode='same')
                notches, _ = find_peaks(-post_peak_smooth, prominence=0.05*np.max(beat_seg))
                notch_rel = notches[0] + systole if len(notches) > 0 else int(systole + 0.3*len(post_peak))
            else:
                notch_rel = int(systole + 0.3*len(post_peak))

            # Detect diastolic peak
            if len(post_peak) > 10:
                diastolic_peak_rel = np.argmax(post_peak[notch_rel-systole:]) + notch_rel
            else:
                diastolic_peak_rel = int(systole + 0.6*(len(beat_seg)-systole))

            # Feature Extraction
            amplitude = beat_seg[systole] - beat_seg[diastole] if systole < len(beat_seg) else 0
            rise_time = max((systole - diastole) / fs, 0.01)
            fall_time = max((notch_rel - systole) / fs, 0.01)
            beat_duration = max((end - start) / fs, 0.01)

            # LDL-specific biomarkers
            SI = amplitude / rise_time
            diastolic_peak_val = beat_seg[diastolic_peak_rel] if diastolic_peak_rel < len(beat_seg) else 0
            AI = ((diastolic_peak_val - beat_seg[systole]) / beat_seg[systole]) * 100 if beat_seg[systole] != 0 else 0
            RI = (diastolic_peak_val / beat_seg[systole]) * 100 if beat_seg[systole] != 0 else 0
            CT = rise_time  # Crest Time is the same as rise time

            baseline = beat_seg[diastole]
            systolic_segment = beat_seg[diastole:systole] - baseline
            diastolic_segment = beat_seg[systole:diastolic_peak_rel] - baseline

            systolic_area = trapezoid(systolic_segment, dx=1/fs) if len(systolic_segment) > 1 else 0
            diastolic_area = trapezoid(diastolic_segment, dx=1/fs) if len(diastolic_segment) > 1 else 0

            # Slope parameters
            upslope = amplitude / rise_time
            downslope = (beat_seg[systole] - beat_seg[notch_rel]) / fall_time if fall_time > 0 else 0

            # Waveform characteristics
            T1 = rise_time
            T2 = max((notch_rel - diastole) / fs, 0.01)
            F1 = T1 / T2

            features = {
                'beat_idx': i,
                'SI': max(SI, 0),
                'AI': np.clip(AI, -100, 100),
                'RI': max(RI, 0),
                'CT': max(CT, 0),
                'amplitude': max(amplitude, 0),
                'upslope': max(upslope, 0),
                'downslope': min(downslope, 0),
                'systolic_area': max(systolic_area, 0),
                'diastolic_area': max(diastolic_area, 0),
                'area_ratio': max(diastolic_area/systolic_area, 0) if systolic_area > 0 else 0,
                'pulse_width': max(beat_duration, 0),
                'T1': max(T1, 0),
                'T2': max(T2, 0),
                'F1': max(F1, 0),
                'heart_rate': 60/beat_duration if beat_duration > 0 else 0
            }

            features_list.append(features)

        except Exception as e:
            print(f"Error processing beat {i}: {str(e)}")
            continue

    return features_list


def process_patient_data(ppg_signal, file_id, ldl_risk):
    beat_features = extract_ppg_features_per_beat(ppg_signal)

    aggregated = {}
    if beat_features:
        for key in beat_features[0].keys():
            values = [beat[key] for beat in beat_features]
            aggregated[f"{key}_mean"] = np.nanmean(values)
            aggregated[f"{key}_std"] = np.nanstd(values)

    aggregated['patient_id'] = file_id
    aggregated['LDL_risk'] = ldl_risk

    return aggregated

In [10]:
all_patients_data = []

for i in range(len(train)):
    if i == 220:
        continue
    file_id = train.iloc[i]['ID']
    label = train.iloc[i]['ЛПНП']
    path = 'ppgs/' + file_id + '.npy'
    ppg = wavelet_denoising(np.load(path))
    patient_data = process_patient_data(ppg, file_id, label)
    all_patients_data.append(patient_data)

df = pd.DataFrame(all_patients_data)
print(f"Dataset shape: {df.shape}")

Dataset shape: (552, 34)


In [16]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.calibration import CalibratedClassifierCV
import joblib


def train_ldl_predictor(df):
    X = df.drop(['patient_id', 'LDL_risk'], axis=1)
    y = df['LDL_risk']

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )

    model = make_pipeline(
        SimpleImputer(strategy='median'),
        StandardScaler(),
        CalibratedClassifierCV(
            RandomForestClassifier(
                n_estimators=200,
                class_weight='balanced',
                random_state=42
            ),
            method='isotonic',
            cv=5
        )
    )

    model.fit(X_train, y_train)

    train_prob = model.predict_proba(X_train)[:, 1]
    test_prob = model.predict_proba(X_test)[:, 1]
    print(f"Train AUC: {roc_auc_score(y_train, train_prob):.3f}")
    print(f"Test AUC: {roc_auc_score(y_test, test_prob):.3f}")

    return model, X.columns.tolist()

trained_model, feature_columns = train_ldl_predictor(df)
joblib.dump((trained_model, feature_columns), 'ldl_predictor.pkl')

Train AUC: 1.000
Test AUC: 0.595


['ldl_predictor.pkl']

In [18]:
def predict_ldl_probability(ppg_signal, model, feature_columns):
    patient_data = process_patient_data(ppg_signal, "new_patient", None)

    features = pd.DataFrame([patient_data])
    features = features[feature_columns]

    nan_count = features.isna().sum().sum()
    quality_flag = nan_count < (0.2 * len(feature_columns))

    prob = model.predict_proba(features)[0, 1]

    return prob, quality_flag


In [21]:
loaded_model, loaded_features = joblib.load('ldl_predictor.pkl')

preds = []
test_ids = []

for i in range(len(test)):
    file_id = test.iloc[i]['ID']
    test_ids.append(file_id)
    path = 'ppgs/' + file_id + '.npy'
    ppg = wavelet_denoising(np.load(path))

    ldl_prob, is_good_quality = predict_ldl_probability(
        ppg,
        loaded_model,
        loaded_features
    )
    preds.append(ldl_prob)


In [24]:
result_df = pd.DataFrame({
    'ID': test_ids,
    'ЛПНП': preds
})

result_df.to_csv('submit.csv', index=False)

In [23]:
result_df.head()

,ID,ЛПНП
0,k31__2__2,0.467705
1,k31__0__0,0.606260
2,k31__13__13,0.302727
3,k31__14__14,0.219106
4,k31__25__25,0.573315
